In [1]:
# you'll want to install torch, transformers, einops, matplotlib and scikit-learn
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer
from einops import rearrange
import numpy as np
import matplotlib.pyplot as plt
from sklearn.manifold import TSNE
from sklearn.linear_model import LogisticRegression


ModuleNotFoundError: No module named 'torch'

In [2]:
# load model and tokenizer
# try out differ
model_name = "distilgpt2"
model = AutoModelForCausalLM.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name)
tokenizer.pad_token = tokenizer.eos_token

NameError: name 'AutoModelForCausalLM' is not defined

In [3]:
# dataset: the same sentence, but ending in a different name, for 20 masculine names and 20 feminine names
prompt_format = "Hi, my name is {}"
man_names = ['John', 'Peter', "Mark", "Paul", "Luke", "Michael", "Valmik", "Mohamed", "Cherif", "Donald", "Yang", "Ruiqi", "Collin", "Jacob", "Guillerme", "Jaime", "Ezra", "Matthew", "Kevin", "Richard"]
woman_names = ["Allie", "Emily", "Frances", "Anisha", "Jingfei", "Fei", "Michelle", "Eunice", "Yaelle", "Britney", "Sarah" , "Leah", "Angelina", "Mary", "Madeleine", "Maria", "Rebecca", "Daphne", "Christine", "Olivia"]
inp_str = [prompt_format.format(name) for name in man_names + woman_names]
labels = len(man_names) * [0] + len(woman_names) * [1]
inp_toks = tokenizer(inp_str, return_tensors="pt", padding=True, truncation=True)

NameError: name 'tokenizer' is not defined

In [4]:
# apply the model to the 20+20 inputs, while saving the hidden state at each layer
out = model(**inp_toks, output_hidden_states=True)

NameError: name 'model' is not defined

In [5]:
# tensor of shape (num_sentences, num_layers, num_tokens, hidden_size)
hidden = torch.stack(out.hidden_states, dim=1)

NameError: name 'torch' is not defined

In [6]:
# challenge: choose a sentence, and get the 5 most likely tokens that are predicted after "Hi, my name is"
# tips: you can use out.logits and the `topk` function. 'Hi, my name is' is common to every sentence, so it doesn't matter which one you choose

In [7]:
# challenge: get the same result, but instead of directly using out.logits, reconstruct it by using the final hidden state and the model's linear decoding layer model.lm_head.weight
# remember to use torch.einsum

In [8]:
# challenge: using the same method as above, get the 5 most likely names that are predicted after "Hi, my name is" by intermediate layers

In [9]:
# for every sentence, get the index of the last token before padding
# this is the token whose embeddings we will study
# because some names are represented with more tokens than others, it's not the same index for every sentence
final_ids = ((inp_toks.input_ids != tokenizer.pad_token_id).sum(1) - 1).tolist()

NameError: name 'inp_toks' is not defined

In [10]:
# get the corresponding hidden states of the token of interest, for every sentence and every layer
last_hidden = torch.stack([hidden[i, :, final_ids[i], :] for i in range(len(final_ids))], dim=0).detach().numpy()
print(last_hidden.shape)

NameError: name 'torch' is not defined

In [11]:
# focus on a single layer, and normalize the hidden states for every sentence
layer = -1 # last layer
X = last_hidden[:,layer,:]
X = (X - X.mean(axis=0)) / X.std(axis=0) # X has shape (num_sentences, hidden_size)

NameError: name 'last_hidden' is not defined

In [12]:
# to visualize, we need to use t-SNE to reduce the dimensionality of the hidden states from hidden_size to 2
X_embedded = TSNE(n_components=2, learning_rate='auto', init='random').fit_transform(X)
fig, ax = plt.subplots()
ax.scatter(X_embedded[:, 0], X_embedded[:, 1], c=labels, )
for i in range(len(inp_str)):
    ax.annotate((man_names + woman_names)[i], (X_embedded[i, 0], X_embedded[i, 1]))
plt.show()

NameError: name 'TSNE' is not defined

In [13]:
# logistic regression to predict gender from the hidden embeddings
train_X = np.concatenate([X[:(len(man_names) // 2)], X[(len(man_names)):(len(man_names) + len(woman_names) // 2)]])
train_Y = labels[:(len(man_names) // 2)] + labels[(len(man_names)):(len(man_names) + len(woman_names) // 2)]


test_X = np.concatenate([X[(len(man_names) // 2):(len(man_names))], X[(len(man_names) + len(woman_names) // 2):(len(man_names) + len(woman_names))]])
test_Y = labels[(len(man_names) // 2):(len(man_names))] + labels[(len(man_names) + len(woman_names) // 2):(len(man_names) + len(woman_names))]

clf = LogisticRegression().fit(train_X, train_Y)
pred_Y = clf.predict(test_X)
accuracy = 1 - np.mean(np.array(pred_Y) - np.array(test_Y))
print(accuracy)

NameError: name 'np' is not defined

In [14]:
# next step: do these visualizations and logistic regressions with the other layers:
# at what layer does the clustering appear?

In [1]:
# project embeddings orthogonal to the logistic regression direction 
# (TODO: check that I multiplied the matrices right)
direction = clf.coef_
dir_proj = direction.T @ direction / np.sum(direction**2)
dir_proj.shape
X_proj = X - X @ dir_proj

NameError: name 'clf' is not defined

In [2]:
# use TSNE to visualize the projected embeddings: how do they cluster?
X_proj_embedded = TSNE(n_components=2, learning_rate='auto', init='random').fit_transform(X_proj)
fig, ax = plt.subplots()
ax.scatter(X_proj_embedded[:, 0], X_proj_embedded[:, 1], c=labels, )
for i in range(len(inp_str)):
    ax.annotate((man_names + woman_names)[i], (X_proj_embedded[i, 0], X_proj_embedded[i, 1]))
plt.show()

NameError: name 'TSNE' is not defined

In [ ]:
# the clusters seem worse, but we can probably make them even worse
# one way to do this would be to iterate: run logistic regression on X_proj, project X_proj on the new direction, and repeat
# try it out!